In [1]:
import numpy as np

# Population parameters reading

In [2]:
def read_population_parameters(file_name):
    params = {}
    f = open(file_name, 'r')
    
    while True:
        # Read line, continue if empty and break if last one
        line = f.readline()
        if not line:
            break
        if line == "":
            continue
        
        # Preprocessing
        line = line.lower()
        
        # Extract parameters
        list_elements = line.split(' ')
        param_name = list_elements[0]
        param_values = []
        for i in range(1, len(list_elements)):
            try:
                param_values.append(float(list_elements[i]))
            except:
                pass
        
        # Add to existing dict
        params[param_name] = param_values
        
    return params

# Individual parameters reading

In [3]:
def add_individual_parameters(line, number_of_params):
    individual_parameters = {}
    
    list_elements = line.split(' ')
    idx_start = 0
    
    for tpl in number_of_params:
        idx_end = idx_start + tpl[1]
        individual_parameters[tpl[0].rstrip()] = [float(list_elements[i].rstrip()) for i in range(idx_start, idx_end)]
        idx_start = idx_end
        
    return individual_parameters

In [4]:
def read_individual_parameters(file_name):
    params = []
    labels = [] 
    number_per_label = []
    individuals = []
    
    f = open(file_name, 'r')
    
    line_count = -1
    
    while True:
        line_count += 1
        # Read line, continue if empty and break if last one
        line = f.readline()
        if not line:
            break
        if line == "":
            continue
            
        # Read line, continue if empty, break if last one
        if (line_count == 0):
            labels = line.split(' ')
            continue
            
        if (line_count == 1):
            number_per_label = line.split(' ')
            
            for idx, name in enumerate(labels):
                params.append((name, int(number_per_label[idx])))
            
            continue
        
        # Add individuals 
        individuals.append(add_individual_parameters(line, params))
        
    return individuals
        

# Read VTK
#### Read distance matrix
#### Get the index of the control points
#### Construct the Kxd and invKd matrices

In [5]:
import scipy.io
mat = scipy.io.loadmat('CorticalSurface.mat')
dist_mat = mat["DistMat"]
data_mesh = mat["data_mesh"]
mesh_points = data_mesh["pts_left"][0][0]
mesh_triangles = data_mesh["tri_left"][0][0]
ind_ctrl = np.hstack(mat["ind_ctrl"])
ind_ctrl = ind_ctrl - 1 # There is a difference within the indexation between matlab and python
print mesh_triangles

[[     1  40965  40963]
 [     1  40963  40966]
 [     1  40966  40968]
 ..., 
 [163842 160927  39991]
 [163114 160927 163842]
 [163114     12 160927]]


In [6]:
def make_matrices(dist_mat, ind_ctrl, bandwidth):
    N_v = dist_mat.shape[0]
    N_c = len(ind_ctrl)
    
    K = np.exp( - dist_mat**2 / bandwidth**2)
    Kd = K[np.ix_(ind_ctrl, ind_ctrl)]
    invKd = np.linalg.inv(Kd)
    
    Kxd = np.zeros((N_v, N_c))
    
    for i in range(0, N_v):
        for j in range(0, N_c):
            Kxd[i, j] = K[i, ind_ctrl[j]]
    
    return [Kxd, invKd]

In [7]:
[Kxd, invKd] = make_matrices(dist_mat, ind_ctrl, 10)

# Compute the needed values to print the signal

In [11]:
def compute_interpolation(Kxd, invKd, input_signal):
    return np.matmul(Kxd, invKd) * input_signal

In [ ]:
def convert_signal(signal, data_mesh):
    indices = data_mesh["indices"]
    Q = data_mesh["Q"]

    output_signal = arrayofsizeNumberOfPoints

    for i, idx in enumerate(indices):
        ### indices where Q == idx
        ### output_signal[indices_where_Q == idx] = signal(i)

    return output_signal

# Compute the parallel curve thanks to the parameters and realization

In [12]:
pop_file = "Work_pop.txt"
indiv_file = "Work_indiv.txt"

pop_params = read_population_parameters(pop_file)
indiv_params = read_individual_parameters(indiv_file)

In [13]:
print pop_params
print indiv_params[0]

{'delta': [-0.00553043, 0.00476241, 0.0080259, 0.0169323, 0.0148448, -0.00402884, -0.00788813, -0.00659071, 0.00985461, 0.00666062, 0.0132312, 0.00776796, 0.0121491, 0.00976731, 0.0139386, 0.0165923, 0.0168574, -0.00881501, 0.00891207, -0.0224051, 0.00643011, 0.00391616, 0.00486719, 0.00232267, 0.00717643, -0.0116141, 0.00430724, 0.00547231, 0.00589756, 0.00124393, 0.00912978, -0.00782982, -0.00438568, -0.00438813, 0.00971957, 0.0188116, -0.00167063, 0.0133241, 0.00977687, 0.00291943, 0.0110046, 0.013348, 0.00943683, 0.00907031, -0.00104016, 0.00698782, 0.00374549, 0.00382106, -0.00656602, -0.00315106, -0.00478631, 0.00377647, 0.00290957, 0.00197268, 0.00394526, 0.00688724, 0.0195074, -0.00252459, -0.00325718, -0.000659537, 0.00449682, 0.000782116, 0.0104009, -0.0048918, 0.0139818, 0.00132042, 0.00687145, 0.00750468, 0.0145722, 0.00492508, 0.0125501, 0.0120501, 0.0092165, 0.00913957, 0.0028359, -0.0052638, 0.00125186, 0.000689806, -0.00694831, -0.0150203, -0.00779999, -0.00680982, -0.0

In [14]:
print min(np.abs(pop_params["delta"]))
print pop_params["delta"][1749]

3.95141e-08
3.95141e-08


In [15]:
def compute_fast_network_signal(pop_params, indiv_params, indiv_number, Kxd, invKd):
    
    #delta = compute_interpolation(Kxd, invKd, pop_params["delta"])
    #nu = compute_interpolation(Kxd, invKd, pop_params["nu"])
    delta = pop_params["delta"]
    nu = pop_params["nu"]
    
    thickness = pop_params["thickness"][0]
    
    
    time_points = np.linspace(60,100,20)
    space_shift = [0 for i in range(0, len(delta))]
    if(indiv_number > -1):
        time_points = np.exp(indiv_params["ksi"][0]) * (time_points - indiv_params["tau"][0])
        space_shift = indiv_params["w"]

    signals = []
    for t in time_points:
        signal = np.divide(space_shift, thickness*np.exp(delta)) + delta + np.divide(nu, thickness * t)
        signal = thickness * np.exp(signal)
        signals.append(signal)
    
    return signals

In [16]:
signals = compute_fast_network_signal(pop_params, indiv_params, -1, Kxd, invKd)

In [17]:
print signals[0]

[ 1.03636188  1.04693479  1.05052966 ...,  1.04671899  1.04601992
  1.04375177]


In [18]:
def compute_meshwork_signal():
    
    return signal

In [19]:
def compute_network_signal():
    
    
    return signal

# Export to visualize in Paraview

In [20]:
signals = compute_fast_network_signal(pop_params, indiv_params, -1, Kxd, invKd)

In [21]:
signals[0]

array([ 1.03636188,  1.04693479,  1.05052966, ...,  1.04671899,
        1.04601992,  1.04375177])

# Write VTK file

In [23]:
import vtk
from vtk import vtkPoints, vtkPolyData, vtkPolyDataWriter, vtkCellArray, vtkPolyLine

def write_vtk_file(mesh_points, mesh_polygones, signal, file_name):
    
    ### Initialize parameters
    nb_points, dimension = mesh_points.shape
    nb_polygones, polygone_dimensions = mesh_polygones.shape
    # TODO : Assert : dimension should be 3
    # TODO : Assert : POints not empty
    # TODO : Assert : Polygone dimension
    # TODO : Assert : polygone not empty
    
    ### Initialize the output file
    points = vtk.vtkPoints()
    vectices = vtk.vtkCellArray()
    
    ### Write the points coordinate
    for pt in mesh_points:
        points.InsertNextPoint((pt[0], pt[1], pt[2]))
    
    ### Write the triangles
    for pl in mesh_polygones:
        triangle = vtk.vtkTriangle()
        triangle.GetPointIds().SetId(0, pl[0])
        triangle.GetPointIds().SetId(1, pl[1])
        triangle.GetPointIds().SetId(2, pl[2])
        vectices.InsertNextCell(triangle)
            
    ### Set up the mesh
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetPolys(vectices)
    
    ### Set up the colors
    colors = vtk.vtkUnsignedCharArray()
    colors.SetNumberOfComponents(3*nb_points)
    colors.SetName("Colors")
    
    for i in range(0, len(mesh_polygones)):
        colors.InsertNextTuple3(255,0,0)
    
    polydata.GetCellData().SetScalars(colors)
    polydata.Modified()

    #### Write the file
    if vtk.VTK_MAJOR_VERSION <= 5:
        polydata.Update()
 
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(file_name)
    if vtk.VTK_MAJOR_VERSION <= 5:
        writer.SetInput(polydata)
    else:
        writer.SetInputData(polydata)

    writer.Write()
    

In [ ]:
write_vtk_file(mesh_points, mesh_triangles, 1, "mesh.vtp")

(163842, 3)

In [151]:
import vtk
from vtk import *
 
#setup points and vertices
Points = vtk.vtkPoints()
Triangles = vtk.vtkCellArray()
 
Points.InsertNextPoint(1.0, 0.0, 0.0)
Points.InsertNextPoint(0.0, 0.0, 0.0)
Points.InsertNextPoint(0.0, 1.0, 0.0)
 
Triangle = vtk.vtkTriangle()
Triangle.GetPointIds().SetId(0, 0)
Triangle.GetPointIds().SetId(1, 1)
Triangle.GetPointIds().SetId(2, 2)
Triangles.InsertNextCell(Triangle)
 
#setup colors
Colors = vtk.vtkUnsignedCharArray()
Colors.SetNumberOfComponents(3)
Colors.SetName("Colors")
Colors.InsertNextTuple3(255,0,0)
Colors.InsertNextTuple3(100,255,0)
 
polydata = vtk.vtkPolyData()
polydata.SetPoints(Points)
polydata.SetPolys(Triangles)
 
polydata.GetPointData().SetScalars(Colors)
polydata.Modified()
if vtk.VTK_MAJOR_VERSION <= 5:
    polydata.Update()
 
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName("TriangleColored.vtp")
if vtk.VTK_MAJOR_VERSION <= 5:
    writer.SetInput(polydata)
else:
    writer.SetInputData(polydata)
 
writer.Write()

1